# HTTP Request/Response Cycle - Lab

## Introduction 

In this lab, we'll make use of the `requests` module commands and properties seen in the previous lesson, to extract information for a web service called **"Open Notify"** to access NASA's space data. 

## Objectives

You will be able to:

* Understand and explain the HTTP Request/Response cycle
* Make http requests in Python using the ‘requests’ library

## Open Notify 

[Open Notify](http://open-notify.org/)  is an an open source project to provide a simple programming interface for some of NASA’s awesome data. This takes live raw data from NASA's systems and turn them into APIs related to space and spacecraft. We can access following information from open notify. 

* Current Location of the International Space Station

* Overhead Pass Predictions for the International Space Station

* Number of People in Space
    
### API endpoints

OpenNotify has several API endpoints. 
>An endpoint is a server route that is used to retrieve different data from the API. 

For example, the `/comments` endpoint on the Reddit API might retrieve information about comments, whereas the `/users` endpoint might retrieve data about users. To access them, you would add the endpoint to the base url of the API.

For the OpenNotify API, we have following end points 

1. Current Location of the International Space Station `/iss-now.json`
2. Overhead Pass Predictions for the International Space Station `/iss-pass.json`    
3. Number of People in Space `/astros.json`

The json extension simple tells us that the data is being returned in a JSON format.

In this lab, we'll be querying a this API to retrieve live data about the International Space Station (ISS). Details on OpenNofity , endpoints, syntax and services it offers can be viewed [Here](http://open-notify.org/Open-Notify-API/)

![](images/iss.jpg)

### Current location of International Space Station

The first endpoint we'll look at on OpenNotify is the` iss-now.json` endpoint (current location of international space station). This endpoint gets the current latitude and longitude of the International Space Station.  Perform following tasks 
* Make a get request to get the latest position of the international space station from the opennotify api's `iss-now` endpoint at http://api.open-notify.org/iss-now.json
* Check the status code of the response
* Interpret the returned code

In [1]:
# You Code Here
import requests
import json
resp = requests.get('http://api.open-notify.org/iss-now.json')
resp.status_code == requests.codes.ok

True

In [2]:
# Your comments 
resp.status_code # code 200 - the request went okay, and result have been returned.

200

* Print the contents of the response and identify its current location

In [3]:
# You Code Here
print(resp.text)

{"message": "success", "iss_position": {"longitude": "-150.1847", "latitude": "-22.3668"}, "timestamp": 1563129895}


In [4]:
data = resp.json()

In [5]:
dict(data)

{'message': 'success',
 'iss_position': {'longitude': '-150.1847', 'latitude': '-22.3668'},
 'timestamp': 1563129895}

In [6]:
import pandas as pd
pd.to_datetime(data['timestamp'], unit='s')

Timestamp('2019-07-14 18:44:55')

In [8]:
# Interpret your results using the API
# On July 14, 2019, ISS will be over following longitude and latitude.
print(data['iss_position']['longitude'], data['iss_position']['latitude'])

-150.1847 -22.3668


### Check the next pass of International space station for a given location

Let's repeat the above for the second endpoint `iss-pass.json`. This end point is used to query the next pass of the space station on a given location. Let's just run as above and record your observations.

In [9]:
# You Code Here
resp = requests.get('http://api.open-notify.org/iss-pass.json')
resp.status_code

400

In [10]:
# Your comments 
# code 400 - bad request

So clearly there is something wrong as we had a 400 response. This is how you should always test your responses for validity. 

if we look at the documentation for the OpenNotify API, we see that the ISS Pass endpoint requires two parameters.

> The ISS Pass endpoint returns when the ISS will next pass over a given location on earth. In order to compute this, we need to pass the coordinates of the location to the API. We do this by passing two parameters -- latitude and longitude.

We can do this by adding an optional keyword argument, params, to our request. In this case, there are two parameters we need to pass:

* lat -- The latitude of the location we want.
* lon -- The longitude of the location we want.

Perform the following tasks :
* Set parameters to reflect the lat and long of New York  (40.71, -74)
* Send a get request to OpenNotify passing in the lat long parameters as k:v pairs in a dictionary
* Check the status code and interpret
* Print the header information and the returned content

In [11]:
# You Code Here
param = {"lat": 40.71, "lon":-74}
resp = requests.get('http://api.open-notify.org/iss-pass.json', params=param)
resp.status_code

200

In [12]:
print(dict(resp.headers))
print(resp.text)

{'Server': 'nginx/1.10.3', 'Date': 'Sun, 14 Jul 2019 18:46:02 GMT', 'Content-Type': 'application/json', 'Content-Length': '519', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}
{
  "message": "success", 
  "request": {
    "altitude": 100, 
    "datetime": 1563128147, 
    "latitude": 40.71, 
    "longitude": -74.0, 
    "passes": 5
  }, 
  "response": [
    {
      "duration": 559, 
      "risetime": 1563154917
    }, 
    {
      "duration": 648, 
      "risetime": 1563160652
    }, 
    {
      "duration": 584, 
      "risetime": 1563166516
    }, 
    {
      "duration": 556, 
      "risetime": 1563172393
    }, 
    {
      "duration": 621, 
      "risetime": 1563178211
    }
  ]
}



In [13]:
data = resp.json()
dict(data)

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1563128147,
  'latitude': 40.71,
  'longitude': -74.0,
  'passes': 5},
 'response': [{'duration': 559, 'risetime': 1563154917},
  {'duration': 648, 'risetime': 1563160652},
  {'duration': 584, 'risetime': 1563166516},
  {'duration': 556, 'risetime': 1563172393},
  {'duration': 621, 'risetime': 1563178211}]}

In [14]:
# Check the API and interpret your results - when will ISS pass over NEW York next ?
# ISS will pass over New York on following times.
risetime = [pd.to_datetime(i['risetime'], unit='s') for i in data['response']]
risetime

[Timestamp('2019-07-15 01:41:57'),
 Timestamp('2019-07-15 03:17:32'),
 Timestamp('2019-07-15 04:55:16'),
 Timestamp('2019-07-15 06:33:13'),
 Timestamp('2019-07-15 08:10:11')]

### Finding the number of people in space

OpenNotify has one more API endpoint, `/astros.json`. It tells you how many people are currently in space. The format of the responses can be studied [HERE](http://open-notify.org/Open-Notify-API/People-In-Space/).

Read the above documentation and perform following tasks:

* Get the response from astros.json endpoint
* Count how many people are currently in space
* List the names of people currently in space.

In [15]:
# You Code Here
resp = requests.get('http://api.open-notify.org/astros.json')
resp.status_code

200

In [16]:
data = resp.json()

In [17]:
data['number']

3

In [18]:
dict(data)

{'people': [{'craft': 'ISS', 'name': 'Alexey Ovchinin'},
  {'craft': 'ISS', 'name': 'Nick Hague'},
  {'craft': 'ISS', 'name': 'Christina Koch'}],
 'number': 3,
 'message': 'success'}

In [19]:
print(dict(resp.headers))
print(resp.text)

{'Server': 'nginx/1.10.3', 'Date': 'Sun, 14 Jul 2019 18:46:19 GMT', 'Content-Type': 'application/json', 'Content-Length': '176', 'Connection': 'keep-alive', 'access-control-allow-origin': '*'}
{"people": [{"craft": "ISS", "name": "Alexey Ovchinin"}, {"craft": "ISS", "name": "Nick Hague"}, {"craft": "ISS", "name": "Christina Koch"}], "number": 3, "message": "success"}


In [20]:
# Interpret the Results - How many people are in space and what are their names 
# There are 3 people in space, and their names are Alexey Ovhinin, Nick Hague, and Christina Koch.

## Summary 

In this lesson we saw how we can use request and response methods to query an Open API. We also saw how to look at the contents returned with the API calls and how to parse them. Next, we'll look at connecting to APIs which are not OPEN, i.e. we would need to pass in some authentication information and filter the results. 